In [2]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

In [3]:
sample = " if you want me"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss:", l, "Prediction:", ''.join(result_str))

0 loss: 2.49672 Prediction:               
1 loss: 2.3542 Prediction:               
2 loss: 2.21767 Prediction:               
3 loss: 2.08072 Prediction:              e
4 loss: 1.86319 Prediction:             me
5 loss: 1.58232 Prediction: i      want me
6 loss: 1.2515 Prediction: if  ou want me
7 loss: 0.951609 Prediction: if you want me
8 loss: 0.696882 Prediction: if you want me
9 loss: 0.517518 Prediction: if you want me
10 loss: 0.35139 Prediction: if you want me
11 loss: 0.249625 Prediction: if you want me
12 loss: 0.152764 Prediction: if you want me
13 loss: 0.0988913 Prediction: if you want me
14 loss: 0.0610778 Prediction: if you want me
15 loss: 0.0361938 Prediction: if you want me
16 loss: 0.0225128 Prediction: if you want me
17 loss: 0.0149441 Prediction: if you want me
18 loss: 0.0105132 Prediction: if you want me
19 loss: 0.00771812 Prediction: if you want me
20 loss: 0.00581666 Prediction: if you want me
21 loss: 0.0044539 Prediction: if you want me
22 loss: 0.00345387